# Grand canonical ensemble transition-matrix Monte Carlo

In this example, flat histogram methods are employed for a small macrostate range from 0 to 5 particles.
To begin, the system is initialized with the minimum number of particles by setting Metropolis acceptance criteria with favorable conditions for adding particles.
The Metropolis criteria are then replaced with the flat histogram criteria.
At this point, typical analysis from the previous tutorials are added.
In addition, we also add checkpoint files, criteria status, and average energy of a given macrostate.
Finally, the simulation is run until the requested number of iterations of the flat histogram algorithm are complete.

A small macrostate range allows the simulation to run quickly with good sampling, and thus it is an ideal starting point to test the simulations. To begin, read the previous SRSW values from file for comparison.

In [1]:
import pandas as pd
import numpy as np

srsw_file = "../test/data/stat150.csv"
fh=pd.read_csv(srsw_file)[:6]
fh['lnPI'] -= np.log(sum(np.exp(fh['lnPI'])))   # renormalize
fh['delta_ln_prob']=fh['lnPI'].diff()
fh

,N,energy,lnPI,energystd,lnPIstd,delta_ln_prob
0,0,-2.312265e-10,-18.707570,6.689238e-10,0.037092,NaN
1,1,-6.057402e-04,-14.037373,6.709198e-10,0.036838,4.670197
2,2,-3.057422e-02,-10.050312,9.649147e-06,0.036964,3.987061
3,3,-8.992832e-02,-6.458921,1.387472e-04,0.037746,3.591391
4,4,-1.784571e-01,-3.145637,3.315245e-05,0.038097,3.313283
5,5,-2.961920e-01,-0.045677,1.348791e-05,0.038458,3.099960


Run a short flat-histogram simulation to compare with these values

In [2]:
script="""
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /feasst/particle/lj.fstprt
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
ThermoParams beta {beta} chemical_potential0 -2.352321
FlatHistogram Macrostate MacrostateNumParticles width 1 min 0 max 5 \
              Bias TransitionMatrix min_sweeps 50
TrialTranslate weight 0.25 tunable_param 1
TrialTransfer weight 1 particle_type 0
Log trials_per_write 1e5 output_file lj.txt
Movie trials_per_write 1e5 output_file lj.xyz
CheckEnergy trials_per_update 1e5 tolerance 1e-8
Tune
CriteriaUpdater trials_per_update 1e5
CriteriaWriter trials_per_write 1e5 output_file lj_fh.txt
Energy output_file lj_en.txt trials_per_update 1 trials_per_write 1e5 multistate true
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001
Run until_criteria_complete true
""".format(beta=1./1.5)

def run(script):
    with open('script.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("~/feasst/build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
    with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# FEASST version: v0.24.1-3-g9284623035-hwh/prefetch_txt2
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /home/hwh/feasst/particle/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 0.6666666666666666 chemical_potential0 -2.352321  
FlatHistogram Bias TransitionMatrix Macrostate MacrostateNumParticles max 5 min 0 min_sweeps 50 width 1  
TrialTranslate tunable_param 1 weight 0.25  
TrialTransfer particle_type 0 weight 1  
Log output_file lj.txt trials_per_write 1e5  
Movie output_file lj.xyz trials_per_write 1e5  
CheckEnergy tolerance 1e-8 trials_per_update 1e5  
Tune  
CriteriaUpdater trials_per_update 1e5  
CriteriaWriter output_file lj_fh.txt trials_per_write 1e5  
Energy multistate true output_file lj_en.txt trials_per_update 1 trials_per_write 1e5  
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001  
Run until_criteria_complete true  
# initializing random number generator with seed: 1701178789
 
 exit: 0


In [3]:
import pandas as pd
import unittest
class TestFlatHistogramLJ(unittest.TestCase):
    """Test flat histogram grand canonical ensemble Monte Carlo simulations"""
    def test_serial_5max(self):
        """Compare the free energies and potential energies with the NIST SRSW
        https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
        https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
        """
        fh2=pd.read_csv('lj_fh.txt', comment='#')
        en=pd.read_csv('lj_en.txt')
        print(fh2[['delta_ln_prob', 'delta_ln_prob_stdev']])
        print('energy', en[['average', 'block_stdev']])
        for macro in range(1, len(fh2)):
            self.assertAlmostEqual(
                fh["delta_ln_prob"][macro],
                fh2["delta_ln_prob"][macro],
                delta=3*fh2["delta_ln_prob_stdev"][macro])
            self.assertAlmostEqual(
                fh['energy'][macro],
                en['average'][macro],
                delta=3*(fh["energystd"][macro]**2 + en['block_stdev'][macro]**2)**(1./2.))
unittest.main(argv=[''], verbosity=2, exit=False)

test_serial_5max (__main__.TestFlatHistogramLJ)
Compare the free energies and potential energies with the NIST SRSW ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


   delta_ln_prob  delta_ln_prob_stdev
0            NaN             0.000000
1        4.66725             0.002131
2        3.98548             0.001933
3        3.59168             0.002015
4        3.31427             0.001937
5        3.09959             0.002238
energy         average   block_stdev
0  1.016696e-16  3.522090e-16
1 -6.057400e-04  7.702390e-10
2 -3.043054e-02  1.836644e-04
3 -9.030030e-02  4.319611e-04
4 -1.779839e-01  7.443080e-04
5 -2.941329e-01  1.343163e-03


A number of files should also have been created.
If the flat histogram method is sampling perfectly, the simulation performs a random walk along the macrostate.
For larger ranges of macrostates, or for more difficult sampling cases, monitoring the macrostate can help you determine what conditions are preventing convergence.
For example, a plot of the macrostate as a function of the number of attempts may look like the following:


In [4]:
pd.read_csv("lj.txt", header=0).dropna(axis='columns')

,volume,num_particles_of_type0,beta,state,energy,LennardJones,LongRangeCorrections,trial,TrialTranslate,tunable,TrialAdd,TrialRemove
0,volume,num_particles_of_type0,beta,state,energy,LennardJones,LongRangeCorrections,trial,TrialTranslate,tunable,TrialAdd,TrialRemove
1,512,5,0.666667,5,-0.14133,-0.126186,-0.0151435,100000,0.925813,2.81401,0.0454977,0.0454591
2,512,2,0.666667,2,-0.00242296,9.95731e-16,-0.00242296,200000,0.939471,2.81401,0.360396,0.362235
3,512,0,0.666667,0,-3.77725e-15,1.60982e-15,-2.49366e-18,300000,0.945475,2.81401,0.505605,0.536263
4,512,2,0.666667,2,-0.0158973,-0.0134743,-0.00242296,400000,0.948501,2.81401,0.581509,0.636591
5,512,3,0.666667,3,-0.00545166,4.16334e-17,-0.00545166,500000,0.950833,2.81401,0.627296,0.699383
6,512,5,0.666667,5,-0.176458,-0.161314,-0.0151435,600000,0.952604,2.81401,0.65805,0.742501
7,512,5,0.666667,5,-0.098881,-0.0837375,-0.0151435,700000,0.953478,2.81401,0.679971,0.773808
8,512,4,0.666667,4,-0.0549583,-0.0452665,-0.00969184,800000,0.954208,2.81401,0.695987,0.797581
9,512,0,0.666667,0,1.60646e-15,-2.77556e-15,-1.6263e-18,900000,0.954738,2.81401,0.708714,0.816365


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!